In [2]:
# This is the code to parse the .spdx file for the SBOMs
# It creates 3 .csv files
# spdx_output.csv: Files independent of packages
# relationships_output.csv: Relationship between differentr packages
# packages_output.csv: Contains different packages

#works from 1.15 to recent

import csv
import glob
import pandas as pd
import linecache

# Define a variable to track the current entry

def cleanup(unwantedTerms,line):
  for i in unwantedTerms:
    line = line.replace(i,"")
  return line


In [2]:
#function to fill in the lists that will later be used to make our 3 csvs
def fill_lists(spdx_file_path):
  with open(spdx_file_path,'r') as spdx_file:
      #counter to keep track of how many dependencies a version has
      #iterate line by line and use "name" as indicator of
      counter = 0
      for line_num,line in enumerate(spdx_file):
        line = line.strip()
        if line.startswith('"name"'):
          #cleanup symbols we do not need
          line = cleanup(['"name":',',','"','"',' '],line)
          #filter out two exceptions and append
          if (line[0:10] == "kubernetes"): versionName = line
          if (line[0:10] != "kubernetes" and line[0:4] != "SBOM"):
            dependencyName.append(line)
            #getting spdxID
            #print(linecache.getline(spdx_file_path,count))
            spdxID.append(cleanup(['"SPDXID":',',','"','"',' '],linecache.getline(spdx_file_path,line_num)))
            #counter keeping track of total amt of dependencies
            counter+=1
        if line.startswith('"versionInfo"'):
          line = cleanup(['"versionInfo":',',','"','"',' '],line)
          #filter out exceptions
          if (line != ""):dependencyVersion.append(line)
      for count in range(counter):
        versionNameList.append(versionName)

In [3]:
def make_csv_dependencies(file_path):
  #csv path for dependencies

  #writing into csv
  with open(file_path,'w') as csv_file:
    fieldnames = ['dependencyName-Version','dependencyName','dependencyVersion','SPDXID']
    csv_writer = csv.DictWriter(csv_file,fieldnames=fieldnames)
    csv_writer.writeheader()
    for count in range(len(dependencyName)):
        csv_writer.writerow({
            'dependencyName-Version': dependencyName[count] + '-' + dependencyVersion[count],
            'dependencyName': dependencyName[count],
            'dependencyVersion': dependencyVersion[count],
            'SPDXID': spdxID[count]
            })

In [4]:
def make_csv_map(file_path):
  #csv path for mapping
  #writing into csv
  with open(file_path,'w') as csv_file:
    fieldnames = ['kubernetesVersion','dependencyName-Version']
    csv_writer = csv.DictWriter(csv_file,fieldnames=fieldnames)
    csv_writer.writeheader()
    for count in range(len(dependencyName)):
        csv_writer.writerow({
            'kubernetesVersion': versionNameList[count],
            'dependencyName-Version': dependencyName[count] + '-' + dependencyVersion[count]
            })

In [5]:
def make_csv_kubeversion(file_path):
  #csv path for kube versions
  #writing into csv
  with open(file_path,'w') as csv_file:
    fieldnames = ['kubernetesVersion']
    csv_writer = csv.DictWriter(csv_file,fieldnames=fieldnames)
    csv_writer.writeheader()
    for count in range(len(versionNameList)):
      csv_writer.writerow({
          'kubernetesVersion': versionNameList[count]

          })

In [6]:
total_files = 0
path ='/content/drive/MyDrive/EC528Colab/JSONsrc/Batch 2/*.*'
for file in glob.glob(path):
  total_files+=1
print(total_files)

119


In [7]:
###RUN THIS TO GENERATE EMPTY

# Create lists to store SPDX information
versionName = None
versionNameList = []
dependencyName = []
dependencyVersion = []
spdxID = []

In [8]:
###RUN THIS TO FILL LISTS

path ='/content/drive/MyDrive/EC528Colab/JSONsrc/Batch 2/*.*'

dependenciescsv_file_path = '/content/drive/MyDrive/EC528Colab/dependencies_entry_2.csv'
mappingcsv_file_path = '/content/drive/MyDrive/EC528Colab/version_dependencies_mapping_2.csv'
kubeversioncsv_file_path = '/content/drive/MyDrive/EC528Colab/version_entry_2.csv'

#iterate through all files in this path
count = 0
for file in glob.glob(path):
  count +=1
  fill_lists(file)
  if(count%10 == 0): print(count/total_files)
  #if(count/total_files >= .5): break


0.08403361344537816
0.16806722689075632
0.25210084033613445
0.33613445378151263
0.42016806722689076
0.5042016806722689
0.5882352941176471
0.6722689075630253
0.7563025210084033
0.8403361344537815
0.9243697478991597


In [9]:
print(len(versionNameList))
print(len(dependencyName))
print(len(dependencyVersion))
print(len(spdxID))

26800
26800
26800
26800


In [10]:
#RUN THIS TO MAKE CSV

make_csv_dependencies(dependenciescsv_file_path)
make_csv_map(mappingcsv_file_path)
make_csv_kubeversion(kubeversioncsv_file_path)


In [11]:
###RUN THIS TO CLEAN

dependenciesDF = pd.read_csv(dependenciescsv_file_path)
mappingDF = pd.read_csv(mappingcsv_file_path)
kubeversionDF = pd.read_csv(kubeversioncsv_file_path)

#function to clean the csv of duplicate rows
dependenciesDF = dependenciesDF.drop_duplicates()
mappingDF = mappingDF.drop_duplicates()
kubeversionDF = kubeversionDF.drop_duplicates()

dependenciesDF.to_csv(dependenciescsv_file_path,encoding='utf-8',index=False)
mappingDF.to_csv(mappingcsv_file_path,encoding='utf-8',index=False)
kubeversionDF.to_csv(kubeversioncsv_file_path,encoding='utf-8',index=False)

In [10]:
###Situational: Run to Merge
#load first set of csv
path_11 = '/content/drive/MyDrive/EC528Colab/dependencies_entry_1.csv'
path_12 = '/content/drive/MyDrive/EC528Colab/version_dependencies_mapping_1.csv'
path_13 = '/content/drive/MyDrive/EC528Colab/version_entry_1.csv'

#load second set of csv
path_21 = '/content/drive/MyDrive/EC528Colab/dependencies_entry_2.csv'
path_22 = '/content/drive/MyDrive/EC528Colab/version_dependencies_mapping_2.csv'
path_23 = '/content/drive/MyDrive/EC528Colab/version_entry_2.csv'
#merge sets of csv
dependenciesDF = pd.concat(
    map(pd.read_csv,[path_11,path_21]), ignore_index=True)
mappingDF = pd.concat(
    map(pd.read_csv,[path_12,path_22]), ignore_index=True)
kubeversionDF = pd.concat(
    map(pd.read_csv,[path_13,path_23]), ignore_index=True)
#write out
dependenciesDF.to_csv('/content/drive/MyDrive/EC528Colab/dependencies_entry.csv',encoding='utf-8',index=False)
mappingDF.to_csv('/content/drive/MyDrive/EC528Colab/version_dependencies_mapping.csv',encoding='utf-8',index=False)
kubeversionDF.to_csv('/content/drive/MyDrive/EC528Colab/version_entry.csv',encoding='utf-8',index=False)